In [4]:
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [17]:
parser = argparse.ArgumentParser(description='Pose Estimation Training')
# parser.add_argument('data', default='./data', metavar='DIR',
#                     help='path to dataset')
# parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
#                     help='number of data loading workers (default: 4)')
parser.add_argument('--epochs', default=50, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
# parser.add_argument('-b', '--batch-size', default=256, type=int,
#                     metavar='N', help='mini-batch size (default: 256)')
parser.add_argument('--lr', '--learning-rate', default=0.1, type=float,
                    metavar='LR', help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M',
                    help='momentum')
parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float,
                    metavar='W', help='weight decay (default: 1e-4)')
parser.add_argument('--print-freq', '-p', default=25, type=int,
                    metavar='N', help='print frequency (default: 25)')
parser.add_argument('--resume', default='', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')
# parser.add_argument('-e', '--evaluate', dest='evaluate', action='store_true',
#                     help='evaluate model on validation set')
# # parser.add_argument('--pretrained', dest='pretrained', action='store_true',
# #                     help='use pre-trained model')
# # parser.add_argument('--world-size', default=1, type=int,
# #                     help='number of distributed processes')
# # parser.add_argument('--dist-url', default='tcp://224.66.41.62:23456', type=str,
# #                     help='url used to set up distributed training')
# # parser.add_argument('--dist-backend', default='gloo', type=str,
# #                     help='distributed backend')
parser.add_argument('--seed', default=None, type=int,
                    help='seed for initializing training. ')
parser.add_argument('--gpu', default=0, type=int,
                    help='GPU id to use.')

_StoreAction(option_strings=['--gpu'], dest='gpu', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help='GPU id to use.', metavar=None)

In [18]:
import scipy.io
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image

class Dataset(torch.utils.data.Dataset):
    """
    The Leeds sport pose dataset.
    (http://sam.johnson.io/research/lsp.html.)
    
    Parameters
    ----------
    split : {'train', 'test'}
    """
    def __init__(self, split):
        self.indices = np.genfromtxt(f'data/lspset_dataset/{split}_indices.csv', delimiter=",").astype(int)
        data = torch.from_numpy(scipy.io.loadmat('data/lspset_dataset/joints.mat')['joints'])
        data = data.permute(2, 0, 1)
        self.joints = data[:, :, :2].reshape(-1, 28)
        self.occlusion_mask = data[:, :, 2].unsqueeze(-1).expand((-1, 14, 2)).reshape(-1, 28)
        self.transform = transform = transforms.Compose(
            [transforms.Resize((220, 220)),
             transforms.ToTensor(),
             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
            ])

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, index):
        index = self.indices[index]
        old_image = Image.open(f'data/lspset_dataset/images/im{index+1:05d}.jpg')
        old_width, old_height = old_image.size
        image = self.transform(old_image)
        
        joints = self.joints[index, :]
        # Make joints in image square 220x220
        joints[0:28:2] *= 220 / old_width
        joints[1:28:2] *= 220 / old_height 
        # Make joints between [0,1]
        joints /= 220
        sample = {'image': image, 'joints':  joints, 
                  'occlusion_mask': self.occlusion_mask[index, :]}
        return sample

In [19]:
# Data
# from data import Dataset
train_loader = torch.utils.data.DataLoader(Dataset('train'), batch_size=32,
                                          shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(Dataset('test'), batch_size=32,
                                          shuffle=True, num_workers=2)

In [20]:
# for i, batch in enumerate(train_loader):
#     input = batch['image'].permute(0,2,3,1) # torch.Size([32, 220, 220, 3])
#     targets = batch['joints'] # torch.Size([32, 28])
#     mask = batch['occlusion_mask'] # torch.Size([32, 28])

In [21]:
%env CUDA_VISIBLE_DEVICES=1
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,96,11,4,padding=(4,4))
        self.bn1 = nn.BatchNorm2d(96)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(96,256,5, padding=(2,2))
        self.bn2 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(256,384,3, padding=(1,1))
        self.conv4 = nn.Conv2d(384,384,3, padding=(1,1))
        self.conv5 = nn.Conv2d(384,256,3)
        self.pool5 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 5 * 5, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 2*14)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
#         print('1: ', x.shape)
        x = self.pool1(x)
        x = self.bn2(F.relu(self.conv2(x)))
#         print('2: ', x.shape)
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
#         print('3: ', x.shape)
        x = F.relu(self.conv4(x))
#         print('4: ', x.shape)
        x = F.relu(self.conv5(x))
#         print('5: ', x.shape)
        x = self.pool5(x)
#         print(x.shape)
        x = x.view(-1, 256 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

env: CUDA_VISIBLE_DEVICES=1


In [22]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [26]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, batch in enumerate(train_loader):
        input = batch['image'] # torch.Size([32, 3, 220, 220])
        target = batch['joints'].float() # torch.Size([32, 28])
        mask = batch['occlusion_mask'].float() # torch.Size([32, 28])
        # measure data loading time
        data_time.update(time.time() - end)

        if args.gpu is not None:
            input = input.cuda()
            target = target.cuda()
            mask = mask.cuda()

        # compute output
        output = model(input)
        loss = criterion(output * mask, target * mask)
        losses.update(loss)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses))


In [27]:
global args, best_prec1
args = parser.parse_args({})

if args.seed is not None:
    random.seed(args.seed)
    torch.manual_seed(args.seed)

if args.gpu is not None:
    warnings.warn('You have chosen a specific GPU. This will completely '
                  'disable data parallelism.')
    torch.device("cuda:"+ str(args.gpu))
    
# args.distributed = args.world_size > 1

# if args.distributed:
#     dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
#                             world_size=args.world_size)

# create model
# if args.pretrained:
#     print("=> using pre-trained model '{}'".format(args.arch))
#     model = models.__dict__[args.arch](pretrained=True)
# else:
print("creating model...")
model = Net()

if args.gpu is not None:
#     model = model.to("cuda:"+ str(args.gpu))
    model = model.cuda()
    
# define loss function (criterion) and optimizer
criterion = nn.MSELoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)

# optionally resume from a checkpoint
if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

cudnn.benchmark = True

/home/x86_64-unknown-linux_ol7-gnu/anaconda-5.2.0/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  if __name__ == '__main__':


creating model...


In [28]:
# if args.evaluate:
#     validate(val_loader, model, criterion)
#     return

for epoch in range(args.start_epoch, args.epochs):
#     if args.distributed:
#         train_sampler.set_epoch(epoch)
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

#     # evaluate on validation set
#     prec1 = validate(val_loader, model, criterion)

#     # remember best prec@1 and save checkpoint
#     is_best = prec1 > best_prec1
#     best_prec1 = max(prec1, best_prec1)
#     save_checkpoint({
#         'epoch': epoch + 1,
#         'arch': args.arch,
#         'state_dict': model.state_dict(),
#         'best_prec1': best_prec1,
#         'optimizer' : optimizer.state_dict(),
#     }, is_best)

Epoch: [0][0/250]	Time 0.327 (0.327)	Data 0.307 (0.307)	Loss 0.2251 (0.2251)	
Epoch: [0][25/250]	Time 0.031 (0.070)	Data 0.003 (0.049)	Loss 0.0308 (0.0970)	
Epoch: [0][50/250]	Time 0.108 (0.067)	Data 0.091 (0.045)	Loss 0.0236 (0.0623)	
Epoch: [0][75/250]	Time 0.030 (0.066)	Data 0.002 (0.044)	Loss 0.0226 (0.0503)	
Epoch: [0][100/250]	Time 0.088 (0.065)	Data 0.071 (0.043)	Loss 0.0258 (0.0439)	
Epoch: [0][125/250]	Time 0.029 (0.065)	Data 0.003 (0.043)	Loss 0.0252 (0.0401)	
Epoch: [0][150/250]	Time 0.090 (0.065)	Data 0.075 (0.043)	Loss 0.0255 (0.0375)	
Epoch: [0][175/250]	Time 0.031 (0.065)	Data 0.003 (0.043)	Loss 0.0258 (0.0356)	
Epoch: [0][200/250]	Time 0.097 (0.065)	Data 0.077 (0.043)	Loss 0.0232 (0.0342)	
Epoch: [0][225/250]	Time 0.031 (0.064)	Data 0.004 (0.043)	Loss 0.0191 (0.0331)	
Epoch: [1][0/250]	Time 0.360 (0.360)	Data 0.343 (0.343)	Loss 0.0255 (0.0255)	
Epoch: [1][25/250]	Time 0.031 (0.070)	Data 0.005 (0.048)	Loss 0.0283 (0.0244)	
Epoch: [1][50/250]	Time 0.104 (0.067)	Data 0.087

Epoch: [10][100/250]	Time 0.052 (0.063)	Data 0.034 (0.044)	Loss 0.0181 (0.0169)	
Epoch: [10][125/250]	Time 0.031 (0.062)	Data 0.003 (0.043)	Loss 0.0185 (0.0169)	
Epoch: [10][150/250]	Time 0.095 (0.063)	Data 0.077 (0.043)	Loss 0.0155 (0.0168)	
Epoch: [10][175/250]	Time 0.031 (0.063)	Data 0.003 (0.043)	Loss 0.0183 (0.0168)	
Epoch: [10][200/250]	Time 0.095 (0.063)	Data 0.077 (0.043)	Loss 0.0175 (0.0168)	
Epoch: [10][225/250]	Time 0.031 (0.063)	Data 0.004 (0.043)	Loss 0.0156 (0.0169)	
Epoch: [11][0/250]	Time 0.346 (0.346)	Data 0.329 (0.329)	Loss 0.0155 (0.0155)	
Epoch: [11][25/250]	Time 0.094 (0.069)	Data 0.077 (0.051)	Loss 0.0159 (0.0167)	
Epoch: [11][50/250]	Time 0.030 (0.065)	Data 0.005 (0.048)	Loss 0.0217 (0.0165)	
Epoch: [11][75/250]	Time 0.077 (0.064)	Data 0.060 (0.045)	Loss 0.0177 (0.0165)	
Epoch: [11][100/250]	Time 0.031 (0.065)	Data 0.003 (0.045)	Loss 0.0170 (0.0164)	


Process Process-64:
Traceback (most recent call last):
  File "/home/x86_64-unknown-linux_ol7-gnu/anaconda-5.2.0/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/x86_64-unknown-linux_ol7-gnu/anaconda-5.2.0/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/x86_64-unknown-linux_ol7-gnu/anaconda-5.2.0/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/x86_64-unknown-linux_ol7-gnu/anaconda-5.2.0/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-18-36e33e179367>", line 37, in __getitem__
    image = self.transform(old_image)
  File "/home/x86_64-unknown-linux_ol7-gnu/anaconda-5.2.0/envs/pytorch/lib/p

KeyboardInterrupt: 

In [ ]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            if args.gpu is not None:
                input = input.cuda(args.gpu, non_blocking=True)
            target = target.cuda(args.gpu, non_blocking=True)

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                       i, len(val_loader), batch_time=batch_time, loss=losses))

    return losses.avg

In [30]:
# # Data loading code
# traindir = os.path.join(args.data, 'train')
# valdir = os.path.join(args.data, 'val')
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])

# train_dataset = datasets.ImageFolder(
#     traindir,
#     transforms.Compose([
#         transforms.Resize(220),
# #         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         normalize,
#     ]))

# # if args.distributed:
# #     train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
# # else:
# train_sampler = None

# train_loader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=args.batch_size, shuffle=(train_sampler is None),
#     num_workers=args.workers, pin_memory=True, sampler=train_sampler)

# val_loader = torch.utils.data.DataLoader(
#     datasets.ImageFolder(valdir, transforms.Compose([
#         transforms.Resize(220),
#         transforms.ToTensor(),
#         normalize,
#     ])),
#     batch_size=args.batch_size, shuffle=False,
#     num_workers=args.workers, pin_memory=True)